In [1]:
# importing libraries and packages
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re
import json

import spacy
nlp = spacy.load('es_core_news_sm')

import warnings
warnings.filterwarnings('ignore')

2022-02-26 00:48:14.787156: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-26 00:48:14.787196: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def accent_rem(name):
    '''
    Function to remove accents from an alphanumeric string:
    params:
        - name: character string.
    Output: string without accents.
    '''
    #Define replacements (possible accents or other special char)
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("ñ", 'n'),
        ("à", "a"),
        ("è", "e"),
        ("ì", "i"),
        ("ò", "o"),
        ("ù", "u"),
        ("ä", 'a'),
        ("ë", "e"),
        ("ï", "i"),
        ("ö", "o"),
        ("ü", "u"),
    )
    #Replace with tuple:
    for a, b in replacements:
        name = name.replace(a, b).replace(a.upper(), b.upper())
    return(name)

def trail_ws(tweet):
    return re.sub(' +', ' ', tweet)

def remove_num(tweet):
    return ''.join([i for i in tweet if not i.isdigit()])

def treat_text(df, text_col, stopw = [], ecolist = [], date_col = 'date', sent_col = 'sentiment'):
        '''
        Function to treat the corpus columns:
        params:
            - df: dataframe to treat.
        Output: Dataframe treated.
        '''
        # Sanity checks:
        df = df.fillna('')
        
        # Formatting corpus columns:

        if date_col == 'date':
            df[date_col] = pd.to_datetime(df[date_col])
        if sent_col == 'sentiment':
            df[sent_col] = df[sent_col].replace(',', '', regex=True)

        #Columns treatment:
        df[text_col] = df[text_col].apply(lambda r: ' '.join([accent_rem(name) for name in r.split()]))
        df[text_col] = df[text_col].apply(lambda r: ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", r).split()))
        df[text_col] = df[text_col].apply(lambda r: r.lower())
        df[text_col] = df[text_col].apply(lambda r: remove_num(r))
        df[text_col] = df[text_col].apply(lambda r: trail_ws(r))
        df[text_col] = df[text_col].apply(lambda r: remove_stops(r, stopw))
        #df[text_col] = df[text_col].apply(lambda r: lemmatize(r))
        df[text_col] = df[text_col].apply(lambda r: ' '.join([accent_rem(name) for name in r.split()]))
    
        # Filter:
        df = df[df[text_col] != '']

        return(df.reset_index(drop=True))

def get_tweets(user, date_ini, date_end, stopw, ecolist):
    '''
    Function to get tweets from a user given a period range.
    params:
        - user: twitter user name.
        - date_ini: first day of time window to retrieve tweets.
        - date_end: last date of time window to retrieve tweets.
    '''
    # Tweets list:
    twts_ls = []

    # Twitter scrapper:
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper('from:' + user + ' since:' + date_ini + ' until:' + date_end).get_items()):
        twts_ls.append([tweet.user.username, tweet.date, tweet.content])
        
    # Tweets dataframe: 
    df = pd.DataFrame(twts_ls, columns=['username', 'date', 'text'])
    df = treat_text(df, 'text', stopw, ecolist, date_col = 'date', sent_col = None)
    return df

In [3]:
path = '../../../../context/SMI/data/utils'
stops = ['/db_stopwords_spanish_1.txt', '/db_stopwords_spanish_2.txt']
stopw = get_stops(stops, path)
ecofile = '/ecofilter.xlsx'
ecofiles = [ecofile]
eco_filter = get_ecofilter(path, ecofiles)

In [54]:
# TODO: 
# Add stop words to database
# Add ecolist to database

In [78]:
# Prueba

user = 'Tallerator'
date_ini = '2015-10-21'
date_end = '2015-10-23'

df_tweets = get_tweets(user, date_ini, date_end, stopw, eco_filter)

In [79]:
df_tweets

,username,date,text
0,Tallerator,2015-10-22 12:53:09+00:00,hora volkswagen investiga motor incorporar sof...
1,Tallerator,2015-10-22 12:35:33+00:00,insolito autocaravana hecha piezas lego funcio...
2,Tallerator,2015-10-22 09:23:43+00:00,volkswagen detiene venta europa coches motor e...
3,Tallerator,2015-10-22 09:15:11+00:00,curioso superb black crystal colaboracion prec...
4,Tallerator,2015-10-22 08:05:04+00:00,vwgate volkswagen utilizo motores trucados ama...
5,Tallerator,2015-10-22 08:03:57+00:00,delorean dmc historia coche regresoalfuturo os...
6,Tallerator,2015-10-21 21:46:54+00:00,ultimahora vwgate volkswagen utilizo motores t...
7,Tallerator,2015-10-21 19:56:35+00:00,noches manana motor
8,Tallerator,2015-10-21 15:28:33+00:00,delorean dmc historia coche regresoalfuturo os...
9,Tallerator,2015-10-21 14:20:24+00:00,lanza espana gti motor


In [69]:
get_ecotweets(df_tweets, eco_filter, 'text')

,username,date,text


In [65]:
prueba = 'inversion publicitaria atresmedia recorta cuota publicitaria mediaset http prnoticias com marketing inversion publicitaria atresmedia mediaset duopolio'

In [66]:
' '.join(list(set(prueba.split())))

'mediaset com http duopolio cuota recorta atresmedia marketing inversion publicitaria prnoticias'

In [88]:
df_tweets['text'] = df_tweets['text'].apply(lambda r: ' '.join(list(set(r.split()))))

In [87]:
' '.join(list(set(df_tweets.iloc[3,2].split())))

'revestidas cristales colaboracion llantas pequenos crystal black preciosa curioso superb'

In [94]:
prueba = 'P'

In [95]:
prueba.split('AGREEMENT')[0]

'P'

In [89]:
df_tweets

,username,date,text
0,Tallerator,2015-10-22 12:53:09+00:00,motor incorporar volkswagen manipulaba hora in...
1,Tallerator,2015-10-22 12:35:33+00:00,lego contamos gt piezas funciona hecha os auto...
2,Tallerator,2015-10-22 09:23:43+00:00,motor via europa ea volkswagen eu coches detie...
3,Tallerator,2015-10-22 09:15:11+00:00,revestidas cristales colaboracion llantas pequ...
4,Tallerator,2015-10-22 08:05:04+00:00,producido argentina amarok motores volkswagen ...
5,Tallerator,2015-10-22 08:03:57+00:00,dmc contamos coche gt and historia os regresoa...
6,Tallerator,2015-10-21 21:46:54+00:00,producido argentina amarok motores volkswagen ...
7,Tallerator,2015-10-21 19:56:35+00:00,noches manana motor
8,Tallerator,2015-10-21 15:28:33+00:00,dmc contamos coche gt and historia os regresoa...
9,Tallerator,2015-10-21 14:20:24+00:00,lanza motor espana gti


In [24]:
prueba = 'consu'

In [3]:
test = 'consumo consumir consumido consume constanza pepe arbol d d'
test.split()

['consumo',
 'consumir',
 'consumido',
 'consume',
 'constanza',
 'pepe',
 'arbol',
 'd',
 'd']

In [26]:
# check ecolist OK:

' '.join([word for word in test.split() if word in prueba])

''

In [2]:
prueba = 'h'
len(prueba)

1

In [5]:
[word for word in test.split() if len(word) > 1]

['consumo', 'consumir', 'consumido', 'consume', 'constanza', 'pepe', 'arbol']

In [10]:
stringliteral = 'hola mi web es https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python'

In [11]:
import re

In [12]:
re.sub(r'http\S+', '', stringliteral)

'hola mi web es '

In [5]:
import json
import pandas as pd

In [116]:
tweet = df_prueba['text'][2]
tweet

'En vigor un nou procediment d’inscripció de naixements i defuncions., i sobre documentació Consulta les novetats http:// bit.ly/1VTv9iA'

In [8]:
def get_stops(stops, path):
    
    with open(path + stops[0]) as f:
        stopw_1 = f.read().splitlines()
    with open(path + stops[1]) as f:
        stopw_2 = f.read().splitlines()

    stopw_1[0] = stopw_1[0].replace('\ufeff', '')
    stopw = stopw_1 + stopw_2
    stopw = [accent_rem(word) for word in stopw]
    return(stopw)

def get_ecofilter(path, files):
    '''
    Function to get the list of words to filter the tweets
    '''
    eco_filter = pd.DataFrame()
    for file in files:
        eco_filter = pd.concat([eco_filter, pd.read_excel(path + file, header=None)], axis=0)
    eco_filter.drop_duplicates(inplace=True)
    eco_filter = eco_filter.iloc[:,0].to_list()
    #eco_filter = [lemmatize(word) for word in eco_filter]
    return [accent_rem(word) for word in eco_filter]

def accent_rem(name):
    '''
    Function to remove accents from an alphanumeric string:
    params:
        - name: character string.
    Output: string without accents.
    '''
    #Define replacements (possible accents or other special char)
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("ñ", 'n'),
        ("à", "a"),
        ("è", "e"),
        ("ì", "i"),
        ("ò", "o"),
        ("ù", "u"),
        ("ä", 'a'),
        ("ë", "e"),
        ("ï", "i"),
        ("ö", "o"),
        ("ü", "u"),
    )
    #Replace with tuple:
    for a, b in replacements:
        name = name.replace(a, b).replace(a.upper(), b.upper())
    return(name)

def tweet_cleaner(tweet, stopw, ecol):
    '''
    Function to treat the text of a tweet.
    params:
        - tweet: the document itself.
    output: the tweet cleaned.
    '''
    # Remove urls (http in advance)
    tweet = re.sub(r'http.*',"", tweet)
    tweet = re.sub(r'pic.twitter\S+', '', tweet)
    # Remove mentions and hastags.
    tweet = re.sub(r'#\S+', '', tweet)
    tweet = re.sub(r'@\S+', '', tweet)
    # Remove spanish vowel accents.
    tweet = ' '.join([accent_rem(word) for word in tweet.split()])
    # Remove special characters.
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", tweet).split())
    # Lower captions.
    tweet = tweet.lower()
    # Remove numbers.
    tweet = ''.join([i for i in tweet if not i.isdigit()])
    # Remove white spaces.
    tweet = re.sub(' +', ' ', tweet)
    # Remove stopwords.
    tweet = ' '.join([word for word in tweet.split() if not word in stopw])
    # Filter words length (<1 and >15).
    tweet = ' '.join([word for word in tweet.split() if len(word) > 1 and len(word) <= 15])
    # Filter ecolist.
    commons = [word for word in ecol if word in tweet]
    if len(commons) < 1:
        tweet = ''
    #if len(tweet) > 1:
        #Lemmatize:
        #tweet = ' '.join([tok.lemma_.lower() for tok in nlp(tweet)])
    return tweet

def treat_text(df, stopw, ecol):
    '''
    Function to treat the tweets of a dataframe:
    params:
        - df: dataframe with tweets.
    output: 
    '''
    df['text'] = df['text'].fillna(' ')
    df['text'] = df['text'].apply(lambda r: tweet_cleaner(r, stopw, ecol))
    df = df[df['text'] != '']
    df = df[['username', 'date', 'text']]
    df = df.reset_index(drop=True)
    
    return df

In [9]:
path = '../../../../context/SMI/data/utils'
stops = ['/db_stopwords_spanish_1.txt', '/db_stopwords_spanish_2.txt']
stopw = get_stops(stops, path)
ecofile = '/ecofilter.xlsx'
ecofiles = [ecofile]
eco_filter = get_ecofilter(path, ecofiles)

In [10]:
path = '../../../../context/SMI/data/get_tweets/2015/DF_2015_10_22.json'
with open(path) as f:
    data = json.load(f)
df_prueba = pd.DataFrame(data)

In [11]:
df_prueba_t = treat_text(df_prueba, stopw, eco_filter)

In [12]:
df_prueba_t

,username,date,text
0,COMBarcelona,2015-10-22 14:00:00,setmana parlarem economia social amb
1,COMBarcelona,2015-10-22 09:10:00,forum inversio healthcare social empreses
2,EqOVER,2015-10-23 00:56:00,buscar maneras mostrar vision mision empresa m...
3,Xaviicastro_,2015-10-22 22:34:00,manda orgullo reina desgracia
4,Xaviicastro_,2015-10-22 22:20:00,mercedes cambiate pareces abestruz
...,...,...,...
86532,OnasisZarate,2015-10-23 01:45:00,encuentro reunion dirigentes prd
86533,OnasisZarate,2015-10-23 00:28:00,quiero felicitar amiga nombramiento presidenta...
86534,OnasisZarate,2015-10-22 15:56:00,amiga felicidades tu cumpleanos gracias tu ami...
86535,OnasisZarate,2015-10-22 04:44:00,lideres izquierda pic twitter com kyyfwhpwv
